# Mike Babb
# babbm@uw.edu
# Find anagrams
## Part 2: Generate and store the anagrams v2.0

In [1]:
# standard libraries - installed by default
import collections
from itertools import product
import pickle
import sqlite3
import string
import os
from time import perf_counter_ns
import timeit

In [2]:
# external libraries - not installed by default
import numpy as np
import pandas as pd

In [3]:
# custom, user-defined functions
import _run_constants as rc
from part_00_file_db_utils import *
from part_00_process_functions import *

### process control flags

In [4]:
# max number of letters to slice to use for the generation of sub-matrices for
# options 5 and 6. More letters means more sub-matrices
# 3 seems to be the sweet spot
n_subset_letters = 3

# set write_data to True to store the generated list of anagrams
write_data = False

### load input data

In [5]:
word_df, wg_df, letter_dict, char_matrix, word_group_id_list, word_id_list, wchar_matrix = load_input_data(db_path = rc.db_path,
                                                                                                           db_name = rc.db_name,
                                                                                                           in_file_path = rc.data_output_file_path)

...loading words into a dataframe...
...query execution took: 1.68 seconds...
...loading word groups into a dataframe...
...query execution took: 1.58 seconds...
...loading the letter dictionary...
...loading the char matrix...
...subsetting the char matrix...


# Test the different matrix splitting techniques

## There are 7 different matrix extraction techniques:
* Option 0: Prepare matrix extraction techniques 1 through 6
* Option 1: No sub-matrices
* Option 2: Word-length - matrices split by the number of characters
* Option 3: First letter - matrices split by each letter
* Option 4: Single-least common letter - matrices split by each letter
* Option 5: n least common letters - matrices split by N least common letters
* Option 6: matrices split by word-length and N least common letters  
See split_matrix() for a more elaborate description.  
test_matrix_extraction_option() invokes the split_matrix() function 7 times to demonstrate the differences in timing and split_matrix() function output.

In [6]:
mat_ext_df = test_matrix_extraction_option(wg_df = wg_df,
                                  letter_dict = letter_dict,
                                  word_group_id_list = word_group_id_list,
                                  wchar_matrix = wchar_matrix,
                                  n_subset_letters = n_subset_letters)

*** matrix extraction option: 0 ***
...enumerating 16,101 records...
...1,000 records enumerated...
...2,000 records enumerated...
...3,000 records enumerated...
...4,000 records enumerated...
...5,000 records enumerated...
...6,000 records enumerated...
...7,000 records enumerated...
...8,000 records enumerated...
...9,000 records enumerated...
...10,000 records enumerated...
...11,000 records enumerated...
...12,000 records enumerated...
...13,000 records enumerated...
...14,000 records enumerated...
...15,000 records enumerated...
...16,000 records enumerated...
...16,101 sub-matrices created...
Total extraction time: 43.01 seconds.
*** matrix extraction option: 1 ***
...0 sub-matrices created...
Total extraction time: 1.19 seconds.
*** matrix extraction option: 2 ***
...enumerating 16,101 records...
...1,000 records enumerated...
...2,000 records enumerated...
...3,000 records enumerated...
...4,000 records enumerated...
...5,000 records enumerated...
...6,000 records enumerated...

In [7]:
# save to sqlite for future refeence
write_data_to_sqlite(df = mat_ext_df, table_name = 'matrix_extraction_technique_timing', 
                     db_path = rc.db_path, db_name = rc.db_name)

...now writing: matrix_extraction_technique_timing


In [8]:
mat_ext_df.head(n=7)

,matrix_extraction_option,n_char_matrix_dict_type,single_letter_matrix_dict_type,letter_selector_matrix_dict_type,nc_ls_matrix_dict_type,n_char_matrix_dict_items,single_letter_matrix_dict_items,letter_selector_matrix_dict_items,nc_ls_matrix_dict_items,process_time
0,0,<class 'dict'>,<class 'dict'>,<class 'dict'>,<class 'dict'>,24,26,2387,16101,43.01
1,1,<class 'NoneType'>,<class 'NoneType'>,<class 'NoneType'>,<class 'NoneType'>,-1,-1,-1,-1,1.19
2,2,<class 'dict'>,<class 'NoneType'>,<class 'NoneType'>,<class 'NoneType'>,24,-1,-1,-1,8.53
3,3,<class 'NoneType'>,<class 'dict'>,<class 'NoneType'>,<class 'NoneType'>,-1,26,-1,-1,8.39
4,4,<class 'NoneType'>,<class 'dict'>,<class 'NoneType'>,<class 'NoneType'>,-1,26,-1,-1,8.65
5,5,<class 'NoneType'>,<class 'NoneType'>,<class 'dict'>,<class 'NoneType'>,-1,-1,2387,-1,8.22
6,6,<class 'NoneType'>,<class 'NoneType'>,<class 'NoneType'>,<class 'dict'>,-1,-1,-1,16101,45.96


In [9]:
# scrolling through this output, it looks options 1-5 take about 10 seconds while
# options 0 and 6 take about 60 seconds. We'll address that later through some profiling and
# using snakeviz. 

In [10]:
# Leaving the matrix extracton blank defaults to option 0: prepare all outputs
wg_df, n_char_matrix_dict, single_letter_matrix_dict, letter_selector_matrix_dict, nc_ls_matrix_dict, p_time = split_matrix(
    letter_dict = letter_dict,
    word_group_id_list = word_group_id_list,
        wg_df = wg_df,
    wchar_matrix = wchar_matrix, 
    n_subset_letters = n_subset_letters    
)

...enumerating 16,101 records...
...1,000 records enumerated...
...2,000 records enumerated...
...3,000 records enumerated...
...4,000 records enumerated...
...5,000 records enumerated...
...6,000 records enumerated...
...7,000 records enumerated...
...8,000 records enumerated...
...9,000 records enumerated...
...10,000 records enumerated...
...11,000 records enumerated...
...12,000 records enumerated...
...13,000 records enumerated...
...14,000 records enumerated...
...15,000 records enumerated...
...16,000 records enumerated...
...16,101 sub-matrices created...
Total extraction time: 39.96 seconds.


In [11]:
# Compute the search space for each matrix extraction option.
# The search space is how many candidates have to be evaluated
# when finding parent/child word relationships. 
# The smaller the search space for a given word, the faster the
# parent/child relationship determination. 
# compute_lookups() calculates this using the number of rows in each of the different matrices and sub-matrices

In [12]:
wg_lu_df = compute_lookups(wg_df=wg_df,
                    n_char_matrix_dict = n_char_matrix_dict,
                    single_letter_matrix_dict = single_letter_matrix_dict,
                    letter_selector_matrix_dict = letter_selector_matrix_dict,
                    nc_ls_matrix_dict = nc_ls_matrix_dict,
                          db_path = rc.db_path, db_name = rc.db_name)

...now writing: word_group_lookup_counts


# demonstrate the different matrix extraction techniques using the word 'achiever'

In [13]:
demo_word = 'achiever'

wg_id = word_df.loc[word_df['lcase'] == demo_word, 'word_group_id'].iloc[0]

demo_wg_df = wg_df.loc[wg_df['word_group_id'] == wg_id, : ]

# option 1 - Full matrix
# No additional data needed

# option 2 -  Number of characters
n_char = demo_wg_df['n_chars'].iloc[0]

# option 3 - First letter
first_letter_id = demo_wg_df['first_letter_id'].iloc[0]

# option 4 - Least common letter
single_letter_id = demo_wg_df['single_letter_id'].iloc[0]

# option 5 - Multiple least common letters
letter_selector_id = demo_wg_df['letter_selector_id'].iloc[0]

# option 6 - Number of characters and multiple least common letters
nc_ls_id = demo_wg_df['nc_ls_id'].iloc[0]


## Demontrate that the different matrix extraction options return identical results

### Select on the full matrix: option 1

In [14]:
# demo the full matrix selection
output = get_values_full_matrix(wg_id = wg_id, 
                    wchar_matrix = wchar_matrix,
                   word_group_id_list = word_group_id_list)

# this is an array of from words to the word 'achiever'
format_demo_output(demo_word = demo_word,
                   word_df = word_df,
                   demo_output = output)

There are 45 parent/from word groups for the word achiever
There are 46 parent/from words for the word achiever
The first five words are:
1588               achiever
14772          archdeceiver
14777         archdetective
14962          architective
15078    archrepresentative
Name: lcase, dtype: object
The last five words are:
225492       urethrovesical
226669        vaucheriaceae
226670       vaucheriaceous
227716    vibrotherapeutics
233687       zepharovichite
Name: lcase, dtype: object


### Select on the matrices split by word-length: option 2

In [15]:
# demo the n char selection
output = get_values_n_char(wg_id = wg_id,
                      n_char = n_char,
                      n_char_matrix_dict = n_char_matrix_dict)

# this is an array of from words to the word 'achiever'
format_demo_output(demo_word = demo_word,
                   word_df = word_df,
                   demo_output = output)

There are 45 parent/from word groups for the word achiever
There are 46 parent/from words for the word achiever
The first five words are:
1588               achiever
14772          archdeceiver
14777         archdetective
14962          architective
15078    archrepresentative
Name: lcase, dtype: object
The last five words are:
225492       urethrovesical
226669        vaucheriaceae
226670       vaucheriaceous
227716    vibrotherapeutics
233687       zepharovichite
Name: lcase, dtype: object


### Select on the matrices split by the first letter: option 3

In [16]:
# demo the first letter selection
output = get_values_single_letter(wg_id = wg_id, single_letter_id = first_letter_id,
                                                           single_letter_matrix_dict = single_letter_matrix_dict)         

# this is an array of from words to the word 'achiever'
format_demo_output(demo_word = demo_word,
                   word_df = word_df,
                   demo_output = output)

There are 45 parent/from word groups for the word achiever
There are 46 parent/from words for the word achiever
The first five words are:
1588               achiever
14772          archdeceiver
14777         archdetective
14962          architective
15078    archrepresentative
Name: lcase, dtype: object
The last five words are:
225492       urethrovesical
226669        vaucheriaceae
226670       vaucheriaceous
227716    vibrotherapeutics
233687       zepharovichite
Name: lcase, dtype: object


### Select on the matrices split by the single least common letter: option 4

In [17]:
# demo the first letter selection
output = get_values_single_letter(wg_id = wg_id, single_letter_id = single_letter_id,
                                                           single_letter_matrix_dict = single_letter_matrix_dict)         

# this is an array of from words to the word 'achiever'
format_demo_output(demo_word = demo_word,
                   word_df = word_df,
                   demo_output = output)

There are 45 parent/from word groups for the word achiever
There are 46 parent/from words for the word achiever
The first five words are:
1588               achiever
14772          archdeceiver
14777         archdetective
14962          architective
15078    archrepresentative
Name: lcase, dtype: object
The last five words are:
225492       urethrovesical
226669        vaucheriaceae
226670       vaucheriaceous
227716    vibrotherapeutics
233687       zepharovichite
Name: lcase, dtype: object


### Select on the matrices split by the letter selector: option 5

In [18]:
# demo with the letter selector
output = get_values_letter_selector(wg_id = wg_id,
                      letter_selector_id = letter_selector_id,
                      letter_selector_matrix_dict = letter_selector_matrix_dict)

# this is an array of from words to the word 'achiever'
format_demo_output(demo_word = demo_word,
                   word_df = word_df,
                   demo_output = output)

There are 45 parent/from word groups for the word achiever
There are 46 parent/from words for the word achiever
The first five words are:
1588               achiever
14772          archdeceiver
14777         archdetective
14962          architective
15078    archrepresentative
Name: lcase, dtype: object
The last five words are:
225492       urethrovesical
226669        vaucheriaceae
226670       vaucheriaceous
227716    vibrotherapeutics
233687       zepharovichite
Name: lcase, dtype: object


### Select on the matrices split by word-length and the letter selector: option 6

In [19]:
# demo with the n_char letter selector
output = get_values_n_char_letter_selector(wg_id = wg_id,
                           nc_ls_id = nc_ls_id,                           
                           nc_ls_matrix_dict=nc_ls_matrix_dict)

# this is an array of from words to the word 'achiever'
format_demo_output(demo_word = demo_word,
                   word_df = word_df,
                   demo_output = output)

There are 45 parent/from word groups for the word achiever
There are 46 parent/from words for the word achiever
The first five words are:
1588               achiever
14772          archdeceiver
14777         archdetective
14962          architective
15078    archrepresentative
Name: lcase, dtype: object
The last five words are:
225492       urethrovesical
226669        vaucheriaceae
226670       vaucheriaceous
227716    vibrotherapeutics
233687       zepharovichite
Name: lcase, dtype: object


In [20]:
# we've tested with one word, let's time many evaluations to get a sense of how quickly 
# the different matrix extraction options work
# use the timeit() function to evaluate how long, on average, a single operation
# takes to complete
# we do this by writing python code encapsulated in quotes which is then sent to the function
# we can store the quoted code in a dictionary and then enumerate. 
# we'll run each code chunk 100 times and then compute the average

n_trials = 100

code_snippet_dict = {
    'Matrix Selection Option 1: Selecting by full matrix':
"""get_values_full_matrix(wg_id = wg_id, wchar_matrix = wchar_matrix, word_group_id_list = word_group_id_list)""",
    'Matrix Selection Option 2: Selecting by word length':
"""get_values_n_char(wg_id = wg_id, n_char = n_char, n_char_matrix_dict = n_char_matrix_dict)""",
    'Matrix Selection Option 3: Selecting by first letter':
"""get_values_single_letter(wg_id = wg_id, single_letter_id = first_letter_id, single_letter_matrix_dict = single_letter_matrix_dict)""",    
    'Matrix Selection Option 4: Selecting by single least common letter':
"""get_values_single_letter(wg_id = wg_id, single_letter_id = single_letter_id, single_letter_matrix_dict = single_letter_matrix_dict)""",
    'Matrix Selection Option 5: Selecting by letter selector':
"""get_values_letter_selector(wg_id = wg_id, letter_selector_id = letter_selector_id, letter_selector_matrix_dict = letter_selector_matrix_dict)""",
    'Matrix Selection Option 6: Selecting by word length and letter selector':
"""get_values_n_char_letter_selector(wg_id = wg_id, nc_ls_id = nc_ls_id, nc_ls_matrix_dict=nc_ls_matrix_dict)"""
}

timing_list = []
for csd, cs in code_snippet_dict.items():    

    # here we time the code execution
    total_time = timeit.timeit(cs, number=n_trials, globals=globals())
    timing_list.append([csd, total_time])

    # total time    
    total_time_formatted = '{:,}'.format(round(total_time, 2))    

    # average time
    avg_time = total_time / n_trials
    avg_time_formatted = '{:,}'.format(round(avg_time, 2)) 
        
    print(csd)
    # average number of seconds per trial
    print('Total time:', total_time_formatted, 'seconds. Average time:', avg_time_formatted, 'seconds.')

    # add a line break to make it easier to read
    print()
    
    

Matrix Selection Option 1: Selecting by full matrix
Total time: 3.43 seconds. Average time: 0.03 seconds.

Matrix Selection Option 2: Selecting by word length
Total time: 2.22 seconds. Average time: 0.02 seconds.

Matrix Selection Option 3: Selecting by first letter
Total time: 1.68 seconds. Average time: 0.02 seconds.

Matrix Selection Option 4: Selecting by single least common letter
Total time: 0.28 seconds. Average time: 0.0 seconds.

Matrix Selection Option 5: Selecting by letter selector
Total time: 0.02 seconds. Average time: 0.0 seconds.

Matrix Selection Option 6: Selecting by word length and letter selector
Total time: 0.02 seconds. Average time: 0.0 seconds.



# Esimate total number of pairs

In [21]:
# how many parent/child relationships are there?
# let's estimate the number of anagrams by assuming that the number of
# parent/from words is a function of word length. 
# estimate_total_pairs() estimates the total number of from/to word pairs
# the reason for estimating the upper bound is that it is both just interesting 
# to know but it also means that we can use the estimated values to allocate an 
# object in memory as opposed to incrementally appending to a list - this is faster
# the object in memory is a NumPy Array that will store integers: from word group id | to word group id

In [22]:
n_possible_anagrams, agg_pos_df = estimate_total_pairs(wg_df = wg_df, letter_selector_matrix_dict = letter_selector_matrix_dict)

...estimated number of from/to pair word pairs: 198,842,245


In [23]:
# save this to disk for future reference
write_data_to_sqlite(df = agg_pos_df, table_name = 'n_possible_anagrams', db_path = rc.db_path, db_name = rc.db_name,
                         if_exists_option='replace',
                         index_option=False, verbose=True)

...now writing: n_possible_anagrams


# Calculate the ratios amongst different timing techniques

In [24]:
# these numbers are orders of magnitude different and therefore hard to interpret.
# let's rescale.

In [25]:
# we'll use product() to compute the cartesian product of the list of timings
# from the timing list, we can compute the ratio of one timing to another
# we can then build a dataframe and cross-tab

expanded_timing_list = []
for me_source, me_target in product(timing_list, repeat=2):    
    # let's unpack this
    me_source_option, me_source_timing = me_source
    me_target_option, me_target_timing = me_target

    me_source_option = me_source_option[17:25]
    me_target_option = me_target_option[17:25]
    me_source_target_timing_ratio = me_source_timing / me_target_timing
    
    # add to the list
    expanded_timing_list.append([me_source_option, me_target_option, me_source_timing, me_target_timing, me_source_target_timing_ratio])    

In [26]:
col_names = ['source', 'target', 'source_timing', 'target_timing', 'timing_ratio']

In [27]:
timing_df = pd.DataFrame(data = expanded_timing_list, columns = col_names)

In [28]:
timing_df.head()

,source,target,source_timing,target_timing,timing_ratio
0,Option 1,Option 1,3.428824,3.428824,1.000000
1,Option 1,Option 2,3.428824,2.217842,1.546018
2,Option 1,Option 3,3.428824,1.682686,2.037709
3,Option 1,Option 4,3.428824,0.283609,12.089959
4,Option 1,Option 5,3.428824,0.016592,206.658968


In [29]:
# use the pd.pivot_table() function to display the ratios
# we computed the product so all cells will be filled
# but we only need to look at the top diagonal
# the bottom diagonal is the inverse of the top diagonal
# from left to right: we can see how much faster option 5 is than option 1
# or how much faster option 4 is than option 3
timing_table = timing_df.pivot_table(index = ['source'], columns = ['target'],
                           values =['timing_ratio']).reset_index(drop = False, names = 'Matrix Extraction')

In [30]:
# use a list comprehension to clean up the column names
col_names = [''.join(cn).replace('timing_ratio', '') for cn in timing_table.columns.tolist()]

In [31]:
timing_table.columns = col_names

In [32]:
timing_table

,Matrix Extraction,Option 1,Option 2,Option 3,Option 4,Option 5,Option 6
0,Option 1,1.000000,1.546018,2.037709,12.089959,206.658968,195.460322
1,Option 2,0.646823,1.000000,1.318037,7.820064,133.671788,126.428245
2,Option 3,0.490747,0.758704,1.000000,5.933114,101.417311,95.921607
3,Option 4,0.082713,0.127876,0.168546,1.000000,17.093438,16.167162
4,Option 5,0.004839,0.007481,0.009860,0.058502,1.000000,0.945811
5,Option 6,0.005116,0.007910,0.010425,0.061854,1.057294,1.000000


In [33]:
# write to sqlite
write_data_to_sqlite(df = timing_table, table_name='matrix_lookup_timing', db_path = rc.db_path, 
                     db_name = rc.db_name)

...now writing: matrix_lookup_timing


In [34]:
# wow. So, based on this analysis of the different 
# matrix extraction options using the word 'achiever':
# options 5 and 6 are the fastest, each over 100 times faster than option 1
# option 4 - just using the single least common letter - is 10X faster than option 1
# and 6X faster than option 3 - using the starting the letter of the word.